In [1]:
from deep_learning_features_audio import *
from deep_learning_dict_api import AudioAnalysisAPI
from deep_learning_dict_datasets import Datasets
import evaluate
from evaluate import load
import pandas as pd
from pathlib import Path
from IPython.display import display, HTML
from torchmetrics import ScaleInvariantSignalNoiseRatio, ScaleInvariantSignalDistortionRatio, SignalNoiseRatio,SignalDistortionRatio, PermutationInvariantTraining, 
from torchmetrics.audio.pesq import PerceptualEvaluationSpeechQuality
from torchmetrics.functional.audio import signal_distortion_ratio




/home/valeriopuglisi/.conda/envs/DLAABE/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# When running this tutorial in Google Colab, install the required packages
# with the following.
# !pip install torchaudio librosa boto3

import torch
import torchaudio
import torchaudio.functional as TAF
import torchaudio.transforms as T

print(torch.__version__)
print(torchaudio.__version__)

1.11.0+cu102
0.11.0+cu102


In [3]:
import io
import os
import math
import tarfile
import multiprocessing

import scipy
import librosa
# import boto3
# from botocore import UNSIGNED
# from botocore.config import Config
import requests
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import time
from IPython.display import Audio, display

[width, height] = matplotlib.rcParams['figure.figsize']
if width < 10:
  matplotlib.rcParams['figure.figsize'] = [width * 2.5, height]

_SAMPLE_DIR = "_sample_data"
SAMPLE_WAV_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/steam-train-whistle-daniel_simon.wav"
SAMPLE_WAV_PATH = os.path.join(_SAMPLE_DIR, "steam.wav")

SAMPLE_WAV_SPEECH_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/VOiCES_devkit/source-16k/train/sp0307/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042.wav"
SAMPLE_WAV_SPEECH_PATH = os.path.join(_SAMPLE_DIR, "speech.wav")

SAMPLE_RIR_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/VOiCES_devkit/distant-16k/room-response/rm1/impulse/Lab41-SRI-VOiCES-rm1-impulse-mc01-stu-clo.wav"
SAMPLE_RIR_PATH = os.path.join(_SAMPLE_DIR, "rir.wav")

SAMPLE_NOISE_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/VOiCES_devkit/distant-16k/distractors/rm1/babb/Lab41-SRI-VOiCES-rm1-babb-mc01-stu-clo.wav"
SAMPLE_NOISE_PATH = os.path.join(_SAMPLE_DIR, "bg.wav")

SAMPLE_MP3_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/steam-train-whistle-daniel_simon.mp3"
SAMPLE_MP3_PATH = os.path.join(_SAMPLE_DIR, "steam.mp3")

SAMPLE_GSM_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/steam-train-whistle-daniel_simon.gsm"
SAMPLE_GSM_PATH = os.path.join(_SAMPLE_DIR, "steam.gsm")

SAMPLE_TAR_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/VOiCES_devkit.tar.gz"
SAMPLE_TAR_PATH = os.path.join(_SAMPLE_DIR, "sample.tar.gz")
SAMPLE_TAR_ITEM = "VOiCES_devkit/source-16k/train/sp0307/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042.wav"

S3_BUCKET = "pytorch-tutorial-assets"
S3_KEY = "VOiCES_devkit/source-16k/train/sp0307/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042.wav"

YESNO_DATASET_PATH = os.path.join(_SAMPLE_DIR, "yes_no")
os.makedirs(YESNO_DATASET_PATH, exist_ok=True)
os.makedirs(_SAMPLE_DIR, exist_ok=True)

def _fetch_data():
  uri = [
    (SAMPLE_WAV_URL, SAMPLE_WAV_PATH),
    (SAMPLE_WAV_SPEECH_URL, SAMPLE_WAV_SPEECH_PATH),
    (SAMPLE_RIR_URL, SAMPLE_RIR_PATH),
    (SAMPLE_NOISE_URL, SAMPLE_NOISE_PATH),
    (SAMPLE_MP3_URL, SAMPLE_MP3_PATH),
    (SAMPLE_GSM_URL, SAMPLE_GSM_PATH),
    (SAMPLE_TAR_URL, SAMPLE_TAR_PATH),
  ]
  for url, path in uri:
    with open(path, 'wb') as file_:
      file_.write(requests.get(url).content)

_fetch_data()

def _download_yesno():
  if os.path.exists(os.path.join(YESNO_DATASET_PATH, "waves_yesno.tar.gz")):
    return
  torchaudio.datasets.YESNO(root=YESNO_DATASET_PATH, download=True)

YESNO_DOWNLOAD_PROCESS = multiprocessing.Process(target=_download_yesno)
YESNO_DOWNLOAD_PROCESS.start()

def _get_sample(path, resample=None):
  effects = [
    ["remix", "1"]
  ]
  if resample:
    effects.extend([
      ["lowpass", f"{resample // 2}"],
      ["rate", f'{resample}'],
    ])
  return torchaudio.sox_effects.apply_effects_file(path, effects=effects)

def get_speech_sample(*, resample=None):
  return _get_sample(SAMPLE_WAV_SPEECH_PATH, resample=resample)

def get_sample(*, resample=None):
  return _get_sample(SAMPLE_WAV_PATH, resample=resample)

def get_rir_sample(*, resample=None, processed=False):
  rir_raw, sample_rate = _get_sample(SAMPLE_RIR_PATH, resample=resample)
  if not processed:
    return rir_raw, sample_rate
  rir = rir_raw[:, int(sample_rate*1.01):int(sample_rate*1.3)]
  rir = rir / torch.norm(rir, p=2)
  rir = torch.flip(rir, [1])
  return rir, sample_rate

def get_noise_sample(*, resample=None):
  return _get_sample(SAMPLE_NOISE_PATH, resample=resample)

def print_stats(waveform, sample_rate=None, src=None):
  if src:
    print("-" * 10)
    print("Source:", src)
    print("-" * 10)
  if sample_rate:
    print("Sample Rate:", sample_rate)
  print("Shape:", tuple(waveform.shape))
  print("Dtype:", waveform.dtype)
  print(f" - Max:     {waveform.max().item():6.3f}")
  print(f" - Min:     {waveform.min().item():6.3f}")
  print(f" - Mean:    {waveform.mean().item():6.3f}")
  print(f" - Std Dev: {waveform.std().item():6.3f}")
  print()
  print(waveform)
  print()

def plot_waveform(waveform, sample_rate, title="Waveform", xlim=None, ylim=None):
  waveform = waveform.numpy()

  num_channels, num_frames = waveform.shape
  time_axis = torch.arange(0, num_frames) / sample_rate

  figure, axes = plt.subplots(num_channels, 1)
  if num_channels == 1:
    axes = [axes]
  for c in range(num_channels):
    axes[c].plot(time_axis, waveform[c], linewidth=1)
    axes[c].grid(True)
    if num_channels > 1:
      axes[c].set_ylabel(f'Channel {c+1}')
    if xlim:
      axes[c].set_xlim(xlim)
    if ylim:
      axes[c].set_ylim(ylim)
  figure.suptitle(title)
  plt.show(block=False)

def plot_specgram(waveform, sample_rate, title="Spectrogram", xlim=None):
  waveform = waveform.numpy()

  num_channels, num_frames = waveform.shape
  time_axis = torch.arange(0, num_frames) / sample_rate

  figure, axes = plt.subplots(num_channels, 1)
  if num_channels == 1:
    axes = [axes]
  for c in range(num_channels):
    axes[c].specgram(waveform[c], Fs=sample_rate)
    if num_channels > 1:
      axes[c].set_ylabel(f'Channel {c+1}')
    if xlim:
      axes[c].set_xlim(xlim)
  figure.suptitle(title)
  plt.show(block=False)

def play_audio(waveform, sample_rate):
  waveform = waveform.numpy()

  num_channels, num_frames = waveform.shape
  if num_channels == 1:
    display(Audio(waveform[0], rate=sample_rate))
  elif num_channels == 2:
    display(Audio((waveform[0], waveform[1]), rate=sample_rate))
  else:
    raise ValueError("Waveform with more than 2 channels are not supported.")

def inspect_file(path):
  print("-" * 10)
  print("Source:", path)
  print("-" * 10)
  print(f" - File size: {os.path.getsize(path)} bytes")
  print(f" - {torchaudio.info(path)}")

def plot_spectrogram(spec, title=None, ylabel='freq_bin', aspect='auto', xmax=None):
  fig, axs = plt.subplots(1, 1)
  axs.set_title(title or 'Spectrogram (db)')
  axs.set_ylabel(ylabel)
  axs.set_xlabel('frame')
  im = axs.imshow(librosa.power_to_db(spec), origin='lower', aspect=aspect)
  if xmax:
    axs.set_xlim((0, xmax))
  fig.colorbar(im, ax=axs)
  plt.show(block=False)

def plot_mel_fbank(fbank, title=None):
  fig, axs = plt.subplots(1, 1)
  axs.set_title(title or 'Filter bank')
  axs.imshow(fbank, aspect='auto')
  axs.set_ylabel('frequency bin')
  axs.set_xlabel('mel bin')
  plt.show(block=False)

def get_spectrogram(
    n_fft = 400,
    win_len = None,
    hop_len = None,
    power = 2.0,
):
  waveform, _ = get_speech_sample()
  spectrogram = T.Spectrogram(
      n_fft=n_fft,
      win_length=win_len,
      hop_length=hop_len,
      center=True,
      pad_mode="reflect",
      power=power,
  )
  return spectrogram(waveform)

def plot_pitch(waveform, sample_rate, pitch):
  figure, axis = plt.subplots(1, 1)
  axis.set_title("Pitch Feature")
  axis.grid(True)

  end_time = waveform.shape[1] / sample_rate
  time_axis = torch.linspace(0, end_time,  waveform.shape[1])
  axis.plot(time_axis, waveform[0], linewidth=1, color='gray', alpha=0.3)

  axis2 = axis.twinx()
  time_axis = torch.linspace(0, end_time, pitch.shape[1])
  ln2 = axis2.plot(
      time_axis, pitch[0], linewidth=2, label='Pitch', color='green')

  axis2.legend(loc=0)
  plt.show(block=False)

def plot_kaldi_pitch(waveform, sample_rate, pitch, nfcc):
  figure, axis = plt.subplots(1, 1)
  axis.set_title("Kaldi Pitch Feature")
  axis.grid(True)

  end_time = waveform.shape[1] / sample_rate
  time_axis = torch.linspace(0, end_time,  waveform.shape[1])
  axis.plot(time_axis, waveform[0], linewidth=1, color='gray', alpha=0.3)

  time_axis = torch.linspace(0, end_time, pitch.shape[1])
  ln1 = axis.plot(time_axis, pitch[0], linewidth=2, label='Pitch', color='green')
  axis.set_ylim((-1.3, 1.3))

  axis2 = axis.twinx()
  time_axis = torch.linspace(0, end_time, nfcc.shape[1])
  ln2 = axis2.plot(
      time_axis, nfcc[0], linewidth=2, label='NFCC', color='blue', linestyle='--')

  lns = ln1 + ln2
  labels = [l.get_label() for l in lns]
  axis.legend(lns, labels, loc=0)
  plt.show(block=False)

DEFAULT_OFFSET = 201
SWEEP_MAX_SAMPLE_RATE = 48000
DEFAULT_LOWPASS_FILTER_WIDTH = 6
DEFAULT_ROLLOFF = 0.99
DEFAULT_RESAMPLING_METHOD = 'sinc_interpolation'

def _get_log_freq(sample_rate, max_sweep_rate, offset):
  """Get freqs evenly spaced out in log-scale, between [0, max_sweep_rate // 2]

  offset is used to avoid negative infinity `log(offset + x)`.

  """
  half = sample_rate // 2
  start, stop = math.log(offset), math.log(offset + max_sweep_rate // 2)
  return torch.exp(torch.linspace(start, stop, sample_rate, dtype=torch.double)) - offset

def _get_inverse_log_freq(freq, sample_rate, offset):
  """Find the time where the given frequency is given by _get_log_freq"""
  half = sample_rate // 2
  return sample_rate * (math.log(1 + freq / offset) / math.log(1 + half / offset))

def _get_freq_ticks(sample_rate, offset, f_max):
  # Given the original sample rate used for generating the sweep,
  # find the x-axis value where the log-scale major frequency values fall in
  time, freq = [], []
  for exp in range(2, 5):
    for v in range(1, 10):
      f = v * 10 ** exp
      if f < sample_rate // 2:
        t = _get_inverse_log_freq(f, sample_rate, offset) / sample_rate
        time.append(t)
        freq.append(f)
  t_max = _get_inverse_log_freq(f_max, sample_rate, offset) / sample_rate
  time.append(t_max)
  freq.append(f_max)
  return time, freq

def plot_sweep(waveform, sample_rate, title, max_sweep_rate=SWEEP_MAX_SAMPLE_RATE, offset=DEFAULT_OFFSET):
  x_ticks = [100, 500, 1000, 5000, 10000, 20000, max_sweep_rate // 2]
  y_ticks = [1000, 5000, 10000, 20000, sample_rate//2]

  time, freq = _get_freq_ticks(max_sweep_rate, offset, sample_rate // 2)
  freq_x = [f if f in x_ticks and f <= max_sweep_rate // 2 else None for f in freq]
  freq_y = [f for f in freq if f >= 1000 and f in y_ticks and f <= sample_rate // 2]

  figure, axis = plt.subplots(1, 1)
  axis.specgram(waveform[0].numpy(), Fs=sample_rate)
  plt.xticks(time, freq_x)
  plt.yticks(freq_y, freq_y)
  axis.set_xlabel('Original Signal Frequency (Hz, log scale)')
  axis.set_ylabel('Waveform Frequency (Hz)')
  axis.xaxis.grid(True, alpha=0.67)
  axis.yaxis.grid(True, alpha=0.67)
  figure.suptitle(f'{title} (sample rate: {sample_rate} Hz)')
  plt.show(block=True)

def get_sine_sweep(sample_rate, offset=DEFAULT_OFFSET):
    max_sweep_rate = sample_rate
    freq = _get_log_freq(sample_rate, max_sweep_rate, offset)
    delta = 2 * math.pi * freq / sample_rate
    cummulative = torch.cumsum(delta, dim=0)
    signal = torch.sin(cummulative).unsqueeze(dim=0)
    return signal

def benchmark_resample(
    method,
    waveform,
    sample_rate,
    resample_rate,
    lowpass_filter_width=DEFAULT_LOWPASS_FILTER_WIDTH,
    rolloff=DEFAULT_ROLLOFF,
    resampling_method=DEFAULT_RESAMPLING_METHOD,
    beta=None,
    librosa_type=None,
    iters=5
):
  if method == "functional":
    begin = time.time()
    for _ in range(iters):
      TAF.resample(waveform, sample_rate, resample_rate, lowpass_filter_width=lowpass_filter_width,
                 rolloff=rolloff, resampling_method=resampling_method)
    elapsed = time.time() - begin
    return elapsed / iters
  elif method == "transforms":
    resampler = T.Resample(sample_rate, resample_rate, lowpass_filter_width=lowpass_filter_width,
                           rolloff=rolloff, resampling_method=resampling_method, dtype=waveform.dtype)
    begin = time.time()
    for _ in range(iters):
      resampler(waveform)
    elapsed = time.time() - begin
    return elapsed / iters
  elif method == "librosa":
    waveform_np = waveform.squeeze().numpy()
    begin = time.time()
    for _ in range(iters):
      librosa.resample(waveform_np, sample_rate, resample_rate, res_type=librosa_type)
    elapsed = time.time() - begin
    return elapsed / iters

In [4]:
def speech_separation_evaluate_metric_with_model_on_librimix(model, dataset, metrics, n_test):
    task = "Speech Separation"
    test_mix_both_file = "test_mix_both_file"
    
    tot_si_snr = torch.zeros(0) 
    tot_si_sdr = torch.zeros(0)
    tot_snr = torch.zeros(0)
    tot_sdr = torch.zeros(0)
    tot_si_pit = torch.zeros(0)
    tot_si_pesq = torch.zeros(0)
    
    references = []
    result = {}
    # os.walk(dataset_path)
    print("dataset: {}".format(dataset))
    # print(Datasets[task])
    print("Datasets[task][dataset] : {}".format(Datasets[task][dataset][test_mix_both_file]))
    test_table = pd.read_table(Datasets[task][dataset][test_mix_both_file], sep=",")
    # display(test_table)
    # cols = test_table.iloc[:,1:4]
    print(len(test_table))
    # display(cols)
    for i, row in enumerate(test_table.iterrows()):
        print("Benchmarking: {}/{}".format(i, test_table.shape[0]))
        # print(i, row[1]['mixture_path'], , , row[1]['noise_path'])
        mixture_path = row[1]['mixture_path']
        source_1_path = row[1]['source_1_path']
        source_2_path = row[1]['source_2_path']
        noise_path = row[1]['noise_path']
        source_1_path_prediction, source_2_path_prediction = AudioAnalysisAPI[model]['function'](audiofile_path=mixture_path)
        # print(source_1_path_prediction, source_2_path_prediction)

        print("==> Mixture : {}".format(mixture_path))
        mixture_waveform, mixture_sample_rate = torchaudio.load(mixture_path)
        # mixture_path_metadata = torchaudio.info(mixture_path)
        # print_stats(mixture_waveform, sample_rate=mixture_sample_rate)
        # plot_waveform(mixture_waveform, mixture_sample_rate)
        # plot_specgram(mixture_waveform, mixture_sample_rate)
        # play_audio(mixture_waveform, mixture_sample_rate)
        
        print("==> source_1 : {}".format(source_1_path))
        source_1_waveform, source_1_sample_rate = torchaudio.load(source_1_path)
        # print_stats(source_1_waveform, sample_rate=source_1_sample_rate)
        # plot_waveform(source_1_waveform, source_1_sample_rate)
        # plot_specgram(source_1_waveform, source_1_sample_rate)
        # play_audio(source_1_waveform, source_1_sample_rate)
        
        print("==> source_2 : {}".format(source_2_path))
        source_2_waveform, source_2_sample_rate = torchaudio.load(source_2_path)
        # print_stats(source_2_waveform, sample_rate=source_2_sample_rate)
        # plot_waveform(source_2_waveform, source_2_sample_rate)
        # plot_specgram(source_2_waveform, source_2_sample_rate)
        # play_audio(source_2_waveform, source_2_sample_rate)
        
        print("==> source_1_prediction : {}".format(source_1_path_prediction))
        source_1_prediction_waveform, source_1_prediction_sample_rate = torchaudio.load(source_1_path_prediction)
        # print_stats(source_1_prediction_waveform, sample_rate=source_1_prediction_sample_rate)
        # plot_waveform(source_1_prediction_waveform, source_1_prediction_sample_rate)
        # plot_specgram(source_1_prediction_waveform, source_1_prediction_sample_rate)
        # play_audio(source_1_prediction_waveform, source_1_prediction_sample_rate)
        
        print("==> source_2_prediction : {}".format(source_2_path_prediction))
        source_2_prediction_waveform, source_2_prediction_sample_rate = torchaudio.load(source_2_path_prediction)
        # print_stats(source_2_prediction_waveform, sample_rate=source_2_prediction_sample_rate)
        # plot_waveform(source_2_prediction_waveform, source_2_prediction_sample_rate)
        # plot_specgram(source_2_prediction_waveform, source_2_prediction_sample_rate)
        # play_audio(source_2_prediction_waveform, source_2_prediction_sample_rate)

        resample_rate_src_1 = max(source_1_sample_rate, source_1_prediction_sample_rate)
        resample_rate_src_2 = max(source_2_sample_rate, source_2_prediction_sample_rate)
        print(resample_rate_src_1, resample_rate_src_2)
        
        # resampling to maximim frequency because is not sure that a model works with the original frequency
        source_1_waveform = TAF.resample(source_1_waveform, source_1_sample_rate, resample_rate_src_1)
        source_2_waveform = TAF.resample(source_2_waveform, source_2_sample_rate, resample_rate_src_2)
        source_1_prediction_waveform = TAF.resample(source_1_prediction_waveform, source_1_prediction_sample_rate, resample_rate_src_1)
        source_2_prediction_waveform = TAF.resample(source_2_prediction_waveform, source_2_prediction_sample_rate, resample_rate_src_2)

        for metric in metrics:
            print("Calculating metric:", metric)
            if metric == "si-snr":
                si_snr = ScaleInvariantSignalNoiseRatio()
                source_1_si_snr = si_snr(source_1_prediction_waveform, source_1_waveform)
                source_2_si_snr = si_snr(source_2_prediction_waveform, source_2_waveform)
                print("{}: {}".format(metric, (source_1_si_snr,source_2_si_snr)))
                result[str(i)+'_'+metric] = {
                    "metric" : metric,
                    "mixture_path": mixture_path,
                    "source_1_path": source_1_path,
                    "source_2_path": source_2_path,
                    "source_1_path_prediction": source_1_path_prediction,
                    "source_2_path_prediction":source_2_path_prediction,
                    "noise_path": noise_path,
                    "source_1_si-snr": str(source_1_si_snr),
                    "source_2_si-snr" : str(source_2_si_snr),
                        
                    }
            if metric == "si-sdr":
                si_sdr = ScaleInvariantSignalDistortionRatio()
                source_1_si_sdr = si_sdr(source_1_prediction_waveform, source_1_waveform)
                source_2_si_sdr = si_sdr(source_2_prediction_waveform, source_2_waveform)
                print("{}: {}".format(metric, (source_1_si_sdr,source_2_si_sdr)))
                result[str(i)+'_'+metric] = {
                    "metric" : metric,
                    "mixture_path": mixture_path,
                    "source_1_path": source_1_path,
                    "source_2_path": source_2_path,
                    "source_1_path_prediction": source_1_path_prediction,
                    "source_2_path_prediction":source_2_path_prediction,
                    "noise_path": noise_path,
                    "source_1_si-snr": str(source_1_si_sdr),
                    "source_2_si-snr" : str(source_2_si_sdr),        
                }
                                

            if metric == "snr":
                snr = SignalNoiseRatio()
                source_1_snr = snr(source_1_prediction_waveform, source_1_waveform)
                source_2_snr = snr(source_2_prediction_waveform, source_2_waveform)
                print("{}: {}".format(metric, (source_1_snr,source_2_snr)))
                result[str(i)+'_'+metric] = {
                    "metric" : metric,
                    "mixture_path": mixture_path,
                    "source_1_path": source_1_path,
                    "source_2_path": source_2_path,
                    "source_1_path_prediction": source_1_path_prediction,
                    "source_2_path_prediction":source_2_path_prediction,
                    "noise_path": noise_path,
                    "source_1_snr": str(source_1_snr),
                    "source_2_snr" : str(source_2_snr),
                    
                }
            if metric == "sdr":
                sdr = SignalDistortionRatio()
                source_1_sdr = sdr(source_1_prediction_waveform, source_1_waveform)
                source_2_sdr = sdr(source_2_prediction_waveform, source_2_waveform)
                print("{}: {}".format(metric, (source_1_sdr,source_2_sdr)))
                result[str(i)+'_'+metric] = {
                    "metric" : metric,
                    "mixture_path": mixture_path,
                    "source_1_path": source_1_path,
                    "source_2_path": source_2_path,
                    "source_1_path_prediction": source_1_path_prediction,
                    "source_2_path_prediction":source_2_path_prediction,
                    "noise_path": noise_path,
                    "source_1_sdr": str(source_1_sdr),
                    "source_2_sdr" : str(source_2_sdr),
                    
                }
            if metric == "pesq":
                nb_pesq = PerceptualEvaluationSpeechQuality(resample_rate_src_1, 'nb')
                wb_pesq = PerceptualEvaluationSpeechQuality(resample_rate_src_1, 'wb')
                source_1_nb_pesq = nb_pesq(source_1_prediction_waveform, source_1_waveform)
                source_2_nb_pesq = nb_pesq(source_2_prediction_waveform, source_2_waveform)

                source_1_wb_pesq = wb_pesq(source_1_prediction_waveform, source_1_waveform)
                source_2_wb_pesq = wb_pesq(source_2_prediction_waveform, source_2_waveform)
                print("{}: {}".format(metric, (source_1_wb_pesq,source_2_wb_pesq)))
                result[str(i)+'_'+metric] = {
                    "metric" : metric,
                    "mixture_path": mixture_path,
                    "source_1_path": source_1_path,
                    "source_2_path": source_2_path,
                    "source_1_path_prediction": source_1_path_prediction,
                    "source_2_path_prediction":source_2_path_prediction,
                    "noise_path": noise_path,
                    "source_1_nb_pesq": str(source_1_nb_pesq),
                    "source_2_nb_pesq" : str(source_2_nb_pesq),
                    "source_1_wb_pesq": str(source_1_wb_pesq),
                    "source_2_wb_pesq" : str(source_2_wb_pesq),
                }   



            if metric == "pit":
                pit = PermutationInvariantTraining(signal_distortion_ratio, 'max')
                source_1_pit = pit(source_1_prediction_waveform, source_1_waveform)
                source_2_pit = pit(source_2_prediction_waveform, source_2_waveform)
                print("{}: {}".format(metric, (source_1_pit,source_2_pit)))
                result[str(i)+'_'+metric] = {
                    "metric" : metric,
                    "mixture_path": mixture_path,
                    "source_1_path": source_1_path,
                    "source_2_path": source_2_path,
                    "source_1_path_prediction": source_1_path_prediction,
                    "source_2_path_prediction":source_2_path_prediction,
                    "noise_path": noise_path,
                    "source_1_pit": str(source_1_pit),
                    "source_2_pit" : str(source_2_pit),
                    
                }
                pass

            
        
        if i == n_test - 1: 
            break

       

    params = {"model": model, "dataset": dataset, "n_test": n_test}
    evaluate.save(path_or_file="./results/", **result, **params)
    
    return result



In [5]:
speech_separation_models = [
    '/api/audioseparation/speech_separation_sepformer_wsj02mix',
    '/api/audioseparation/speech_separation_sepformer_wsj03mix'
    ]
speech_separation_dataset = "Libri2Mix16kMin"
metrics = ["si-snr", "si-sdr", "pesq"]

speech_separation_evaluate_metric_with_model_on_librimix(speech_separation_models[0], speech_separation_dataset, metrics, 1 )


dataset: Libri2Mix16kMin
Datasets[task][dataset] : /storage/data_8T/datasets/audio/LibriMix/Libri2Mix/wav16k/min/metadata/mixture_test_mix_both.csv
3000
Benchmarking: 0/3000
Resampling the audio from 16000 Hz to 8000 Hz
==> Mixture : /storage/data_8T/datasets/audio/LibriMix/Libri2Mix/wav16k/min/test/mix_both/4077-13754-0001_5142-33396-0065.wav
==> source_1 : /storage/data_8T/datasets/audio/LibriMix/Libri2Mix/wav16k/min/test/s1/4077-13754-0001_5142-33396-0065.wav
==> source_2 : /storage/data_8T/datasets/audio/LibriMix/Libri2Mix/wav16k/min/test/s2/4077-13754-0001_5142-33396-0065.wav
==> source_1_prediction : /storage/data_itcoin/svoice_space/DeepLearningAudioAnalyzer/audio-be/separation_sepformer_wsj02mix/SpeechSeparationSepformerWsj02mix_source1_4077-13754-0001_5142-33396-0065.wav
==> source_2_prediction : /storage/data_itcoin/svoice_space/DeepLearningAudioAnalyzer/audio-be/separation_sepformer_wsj02mix/SpeechSeparationSepformerWsj02mix_source2_4077-13754-0001_5142-33396-0065.wav
16000 

/home/valeriopuglisi/.conda/envs/DLAABE/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (ScaleInvariantSignalNoiseRatio). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
                `from torchmetrics.utilities import check_forward_full_state_property`
                that can be used to check if the `full_state_update=True` (old and potential slower behaviour,
                default for now) or if `full_state_update=False` can be used safely.
                
  warnings.warn(*args, **kwargs)


pesq: (tensor(1.4443), tensor(1.6474))


{'0_si-snr': {'metric': 'si-snr',
  'mixture_path': '/storage/data_8T/datasets/audio/LibriMix/Libri2Mix/wav16k/min/test/mix_both/4077-13754-0001_5142-33396-0065.wav',
  'source_1_path': '/storage/data_8T/datasets/audio/LibriMix/Libri2Mix/wav16k/min/test/s1/4077-13754-0001_5142-33396-0065.wav',
  'source_2_path': '/storage/data_8T/datasets/audio/LibriMix/Libri2Mix/wav16k/min/test/s2/4077-13754-0001_5142-33396-0065.wav',
  'source_1_path_prediction': '/storage/data_itcoin/svoice_space/DeepLearningAudioAnalyzer/audio-be/separation_sepformer_wsj02mix/SpeechSeparationSepformerWsj02mix_source1_4077-13754-0001_5142-33396-0065.wav',
  'source_2_path_prediction': '/storage/data_itcoin/svoice_space/DeepLearningAudioAnalyzer/audio-be/separation_sepformer_wsj02mix/SpeechSeparationSepformerWsj02mix_source2_4077-13754-0001_5142-33396-0065.wav',
  'noise_path': '/storage/data_8T/datasets/audio/LibriMix/Libri2Mix/wav16k/min/test/noise/4077-13754-0001_5142-33396-0065.wav',
  'source_1_si-snr': 'tensor(